<a href="https://colab.research.google.com/github/tbhova/Coursera-Geometric-Algorithms/blob/master/ComputationalGeometryWeek1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week1

## Excercise 1.1

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from typing import List
from typing import Tuple
from enum import Enum
import unittest
import numpy as np
import math

class Turn(Enum):
  LEFT = 1
  RIGHT = 2
  ON_SEGMENT = 3
  ON_LINE = 4  # Colinear with the segment but not on the segment

class Intersection(Enum):
  INTERSECTS = 1
  NO_INTERSECTS = 2
  ON_BORDER = 3

class PolygonContainment(Enum):
  INSIDE = 1
  OUTSIDE = 2
  ON_BORDER = 3
    
class Point:
  def __init__(self, x = 0, y = 0):
    self.x = x
    self.y = y

  x = 0
  y = 0
    
  def ToString(self):
      return str(self.x) + ", " + str(self.y)
  def __repr__(self):
      return self.ToString()
  def __str__(self):
      return self.ToString()
  def __eq__(self, other):
    return self.ToString() == other.ToString()
  def __hash__(self):
    return hash(self.ToString())
    
class Line:
  def __init__(self, p1 = Point(), p2 = Point()):
    self.p1 = p1
    self.p2 = p2
    
  p1 = Point()
  p2 = Point()
    
  def ToString(self):
    return "(" + self.p1.ToString() + "), (" + self.p2.ToString() + ")"       
  def __repr__(self):
    return self.ToString()
  def __str__(self):
    return self.ToString()

class Triangle:
  def __init__(self, a = Point(), b = Point(), c = Point()):
    self.a = a
    self.b = b
    self.c = c
    
  a = Point()
  b = Point()
  c = Point()
    
  def ToString(self):
    return "(" + self.a.ToString() + "), (" + self.b.ToString() + "), (" + self.c.ToString() + ")"       
  def __repr__(self):
    return self.ToString()
  def __str__(self):
    return self.ToString()
    
class Polygon:
  def __init__(self, vertices = List[Point]):
    self.vertices = vertices
    
  # Counterclockwise vertices of the polygon
  vertices = List[Point]
    
  def ToString(self):
    out = ""
    for p in vertices:
      out += "(" + p.ToString() + "), "
    return out
    
  def __repr__(self):
    return self.ToString()
  def __str__(self):
    return self.ToString()


In [0]:
def between(e1, e2, test):
    return min(e1, e2) <= test and test <= max(e1, e2)

def FindTurn(line : Line, point : Point) -> Turn:
    m = np.array([[line.p1.x, line.p1.y, 1], [line.p2.x, line.p2.y, 1], [point.x, point.y, 1]])
    det = np.linalg.det(m)
    if abs(det - 0) < 1e-10:
        if between(line.p1.x, line.p2.x, point.x) and between(line.p1.y, line.p2.y, point.y):
            return Turn.ON_SEGMENT
        return Turn.ON_LINE
    elif det > 0:
        return Turn.LEFT
    return Turn.RIGHT

In [33]:
class TestNotebook(unittest.TestCase):

  def test_find_turn_example_1(self):
    l = Line(Point(-4, 4), Point(2, -2))
    p = Point(0, 0)
    self.assertEqual(FindTurn(l, p), Turn.ON_SEGMENT)
    
    p = Point(5, -5)
    self.assertEqual(FindTurn(l, p), Turn.ON_LINE)
    
    p = Point(1, 1)
    self.assertEqual(FindTurn(l, p), Turn.LEFT)
    
    p = Point(-1, -1)
    self.assertEqual(FindTurn(l, p), Turn.RIGHT)
    
  def test_find_turn_example_2(self):
    l = Line(Point(-4, 0), Point(2, -10))
    
    p = Point(-8, 4)
    self.assertEqual(FindTurn(l, p), Turn.RIGHT)
    
    p = Point(2, -7)
    self.assertEqual(FindTurn(l, p), Turn.LEFT)
    
    p = Point(9, 3)
    self.assertEqual(FindTurn(l, p), Turn.LEFT)
    
    p = Point(-4, 6)
    self.assertEqual(FindTurn(l, p), Turn.LEFT)
    
    p = Point(-8, -5)
    self.assertEqual(FindTurn(l, p), Turn.RIGHT)
    
    p = Point(7, 1)
    self.assertEqual(FindTurn(l, p), Turn.LEFT)
    
    p = Point(2, 4)
    self.assertEqual(FindTurn(l, p), Turn.LEFT)
    
    p = Point(-3, -9)
    self.assertEqual(FindTurn(l, p), Turn.RIGHT)
        
  def test_find_turn_additional(self):
    # Vertical line
    l = Line(Point(0,0), Point(0,6))
    p = Point(0,3)
    self.assertEqual(FindTurn(l, p), Turn.ON_SEGMENT)
    
    p = Point(3,3)
    self.assertEqual(FindTurn(l, p), Turn.RIGHT)
    
    p = Point(-1,3)
    self.assertEqual(FindTurn(l, p), Turn.LEFT)
    
    l = Line(Point(0,6), Point(0,0))
    p = Point(3,3)
    self.assertEqual(FindTurn(l, p), Turn.LEFT)
    
    
    l = Line(Point(-3, -1), Point(-3, 4))
    p = Point(0, 0)
    self.assertEqual(FindTurn(l, p), Turn.RIGHT)

unittest.main(argv=[''], verbosity=2, exit=False)

test_find_turn_additional (__main__.TestNotebook) ... ok
test_find_turn_example_1 (__main__.TestNotebook) ... ok
test_find_turn_example_2 (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK


## Excercise 1.2

In [0]:
def TriangleViaTurn(t : Triangle, p : Point) -> PolygonContainment:
  turns = []
  # Following the triangle in 1 direction should result in all left or all right turns
  # (depending on the triange) if the point is inside.
  turns.append(FindTurn(Line(t.a, t.b), p))
  turns.append(FindTurn(Line(t.b, t.c), p))
  turns.append(FindTurn(Line(t.c, t.a), p))
  if Turn.ON_SEGMENT in turns:
      return PolygonContainment.ON_BORDER
  if all(elem == Turn.LEFT for elem in turns) or all(elem == Turn.RIGHT for elem in turns):
      return PolygonContainment.INSIDE
  return PolygonContainment.OUTSIDE

# https://www.youtube.com/watch?v=HYAgJN3x4GA
def TriangleViaVectors(t : Triangle, p : Point) -> PolygonContainment:
  w1 = (t.a.x * (t.c.y - t.a.y) + (p.y - t.a.y) * (t.c.x-t.a.x) - p.x * (t.c.y - t.a.y)) / (
      (t.b.y-t.a.y) * (t.c.x - t.a.x) - (t.b.x-t.a.x) * (t.c.y - t.a.y))
  w2 = (p.y - t.a.y - w1 * (t.b.y - t.a.y)) / (t.c.y-t.a.y)
    
  if (abs(w1 - 0) < 1e-10 or abs(w2 - 0) < 1e-10) and w1 <= 1 and w2 <= 1:
      return PolygonContainment.ON_BORDER
  # On side opposite A
  if w1 > 0 and w2 > 0 and abs(w1 + w2 - 1) < 1e-10:
      return PolygonContainment.ON_BORDER
  if w1 > 0 and w2 > 0 and w1 + w2 < 1:
      return PolygonContainment.INSIDE
  return PolygonContainment.OUTSIDE

def InsideTriangle(triangle : Triangle, point : Point) -> PolygonContainment:
#   return TriangleViaTurn(triangle, point)
  return TriangleViaVectors(triangle, point)

In [35]:
class TestNotebook(unittest.TestCase):

  def test_inside_triangle_1(self):
    t = Triangle(Point(0, 0), Point(5, 5), Point(0, 6))
    t2 = Triangle(Point(0, 0), Point(0, 6), Point(5, 5)) # Reverse area direction
    
    p = Point(3, 3)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.ON_BORDER)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.ON_BORDER)
    
    p = Point(6, 6)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.OUTSIDE)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.OUTSIDE)
    
    p = Point(2, 3)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.INSIDE)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.INSIDE)
    
  def test_inside_triangle_2(self):
    t = Triangle(Point(1, 0), Point(0, 3), Point(3, 3))
    t2 = Triangle(Point(1, 0), Point(3, 3), Point(0, 3)) # Reverse area direction
    
    p = Point(0, 0)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.OUTSIDE)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.OUTSIDE)
    
    p = Point(0, 1)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.OUTSIDE)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.OUTSIDE)
    
    p = Point(0, 2)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.OUTSIDE)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.OUTSIDE)
    
    p = Point(0, 3)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.ON_BORDER)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.ON_BORDER)
    
    p = Point(1, 0)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.ON_BORDER)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.ON_BORDER)
    
    p = Point(1, 1)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.INSIDE)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.INSIDE)
    
    p = Point(1, 2)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.INSIDE)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.INSIDE)
    
    p = Point(1, 3)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.ON_BORDER)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.ON_BORDER)
    
    p = Point(2, 0)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.OUTSIDE)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.OUTSIDE)
    
    p = Point(2, 1)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.OUTSIDE)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.OUTSIDE)
    
    p = Point(2, 2)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.INSIDE)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.INSIDE)
    
    p = Point(2, 3)
    self.assertEqual(InsideTriangle(t, p), PolygonContainment.ON_BORDER)
    self.assertEqual(InsideTriangle(t2, p), PolygonContainment.ON_BORDER)

unittest.main(argv=[''], verbosity=2, exit=False)

test_inside_triangle_1 (__main__.TestNotebook) ... ok
test_inside_triangle_2 (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


## Exercise 1.3 and 1.4

In [0]:
# Returns true if the ray eminating to the left (negative x) of p inserects the line segment l.
def RayIntersects(l : Line, p : Point) -> Intersection:
  # The ray insersects if the ray is between the segment's endpoints in y
  if not between(l.p1.y, l.p2.y, p.y):
      return False 
  # AND p is to the right of l
  # We need p1 below p2 to test for right turns
  l2 = l
  if l.p1.y > l.p2.y:
      l2 = Line(l.p2, l.p1)
  turn = FindTurn(l2, p)
  if turn == Turn.RIGHT:
    return Intersection.INTERSECTS
  elif turn == Turn.ON_SEGMENT:
    return Intersection.ON_BORDER
  return Intersection.NO_INTERSECTS
    
def InsidePolygon(poly : Polygon, q : Point) -> PolygonContainment :
  intersections = 0
  
  for i in range(len(poly.vertices)):
    l = Line(poly.vertices[i - 1], poly.vertices[i])
    intersection = RayIntersects(l, q)
    if intersection == Intersection.INTERSECTS:
      intersections += 1
    elif intersection == Intersection.ON_BORDER:
      return PolygonContainment.ON_BORDER
  
  return PolygonContainment.INSIDE if (intersections % 2 == 1) else PolygonContainment.OUTSIDE

In [37]:
class TestNotebook(unittest.TestCase):

  def test_inside_polygon_1(self):
    poly = Polygon([Point(-3, -1), Point(3, -1), Point(3, 5), Point(0, 2), Point(-3, 4)])

    q = Point(1, 3)
    self.assertEqual(InsidePolygon(poly, q), PolygonContainment.ON_BORDER)
      
    q = Point(-1, 3)
    self.assertEqual(InsidePolygon(poly, q), PolygonContainment.OUTSIDE)
      
    q = Point(0, 0)
    self.assertEqual(InsidePolygon(poly, q), PolygonContainment.INSIDE)
      
    q = Point(-2, 3)
    self.assertEqual(InsidePolygon(poly, q), PolygonContainment.INSIDE)
      
  def test_exercise_1_4(self):
    poly = Polygon([Point(-2, -3), Point(1, -4), Point(3, -2), Point(2, 1), Point(-2, 1)])
      
    q = Point(2, -3)
    self.assertEqual(InsidePolygon(poly, q), PolygonContainment.ON_BORDER)
      
    q = Point(3, 0)
    self.assertEqual(InsidePolygon(poly, q), PolygonContainment.OUTSIDE)
      
    q = Point(0, 0)
    self.assertEqual(InsidePolygon(poly, q), PolygonContainment.INSIDE)
      
    q = Point(2, -1)
    self.assertEqual(InsidePolygon(poly, q), PolygonContainment.INSIDE)
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_exercise_1_4 (__main__.TestNotebook) ... ok
test_inside_polygon_1 (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


# Week 2
## 2.1 Convex polygon

In [0]:
def IsConvex(poly : Polygon):
  if (len(poly.vertices) < 3):
    return False
  for i in range(len(poly.vertices)):
    if (FindTurn(Line(poly.vertices[i-2], poly.vertices[i-1]), poly.vertices[i]) != Turn.LEFT):
      return False
  return True

In [39]:
class TestNotebook(unittest.TestCase):

  def test_is_convex(self):
    convex_poly = Polygon([Point(-2, -3), Point(1, -4), Point(3, -2), Point(2, 1), Point(-2, 1)])
    self.assertEqual(IsConvex(convex_poly), True)

    nonconvex_poly = Polygon([Point(-3, -1), Point(3, -1), Point(3, 5), Point(0, 2), Point(-3, 4)])     
    self.assertEqual(IsConvex(nonconvex_poly), False)

unittest.main(argv=[''], verbosity=2, exit=False)

test_is_convex (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## Build Convex Hull

In [0]:
def Distance(line : Line, p : Point):
  # https://en.wikipedia.org/wiki/Distance_from_a_point_to_a_line#Line_defined_by_two_points
  numerator = abs((line.p2.y - line.p1.y)*p.x - (line.p2.x - line.p1.x)*p.y - + line.p2.x * line.p1.y - line.p2.y * line.p1.x)
  denominator = sqrt(((line.p2.y - line.p1.y) ** 2) + ((line.p2.x - line.p1.x) ** 2))
  return numerator/denominator

#def CheckTriangle(tri, points: list[Point]) -> List[Point]:
#  tri = Triangle(line.p1, line.p2)
#
#  has_max = False
#  max_distance = 0
#  max = Point(0,0)
#  for p in points:
#    if (InsideTriangle(tri, p)):
#      has_max = True
#      if (Distance(Line(tri))):

def Angle(p1: Point, p2: Point):
  return (np.rad2deg(np.arctan2(p2.y-p1.y, p2.x-p1.x)) + 360) % 360

def BuildConvexHull(points : List[Point]) -> Polygon:
  # Quick hull
  if (len(points) < 3):
    return Polygon(points);
  leftmost = min(points, key = lambda p: p.x)
  rightmost = max(points, key = lambda p: p.x)
  topmost = max(points, key = lambda p: p.y)
  bottommost = min(points, key = lambda p: p.y)

  poly = []
  poly.append(bottommost)

  remaining_points = points
  remaining_points.remove(bottommost)
  angle = 0
  last_point = bottommost
  iter = 0

  while (len(remaining_points) > 0):
    #print(last_point.ToString())
    next_point = min(remaining_points, key = lambda p: (Angle(last_point, p) - angle + 360) % 360)
    if (next_point == bottommost):
      break
    angle = Angle(last_point, next_point)
    #print(angle)
    last_point = next_point
    # Normalize to remove duplicate points forming a straight line.
    if (len(poly) >= 2):
      if (FindTurn(Line(poly[-2], poly[-1]), next_point) == Turn.ON_LINE):
        poly.pop(-1)
    poly.append(next_point)
    remaining_points.remove(last_point)
    if (iter == 0):
      remaining_points.append(bottommost)
    iter += 1

  # See if last point forms a straight line.
  if (FindTurn(Line(poly[-2], poly[-1]), poly[0]) == Turn.ON_LINE):
    poly.pop(-1)


  return Polygon(poly);

In [41]:
class TestNotebook(unittest.TestCase):

  def test_build_convex_hull(self):
    points = [Point(-10, 2), Point(-7, 2), Point(-4, 3), Point(-6, 5), Point(-8, 7), Point(-2, 8), Point(2, 3), Point(4, 5), Point(6, 1), Point(7, -5), Point(2, -7), Point(-4, -4), Point(-8, -6), Point(-7, -1), Point(2, 6)]
    expected = Polygon([Point(-10, 2), Point(-8, -6), Point(2, -7), Point(7, -5), Point(6, 1), Point(4, 5), Point(-2, 8), Point(-8, 7)])
    actual = BuildConvexHull(points)
    self.assertEqual(len(actual.vertices), len(expected.vertices))
    #for i in range(len(actual.vertices)):
    # self.assertEqual(actual.vertices[i] == expected.vertices[i], True)
    self.assertCountEqual(actual.vertices, expected.vertices)
    
  def test_build_convex_hull2(self):
    points = [Point(-3, 11), Point(-7, 10), Point(-9, 7), Point(-10, 0), Point(-11, -12), Point(4, -6), Point(6, -5), Point(7, 4), Point(6, 7), Point(4, 9)]
    expected = Polygon([Point(4, -6), Point(6, -5), Point(7, 4), Point(6, 7), Point(4, 9), Point(-3, 11), Point(-7, 10), Point(-9, 7), Point(-10, 0), Point(-11, -12)])
    actual = BuildConvexHull(points)
    self.assertEqual(len(actual.vertices), len(expected.vertices))
    self.assertCountEqual(actual.vertices, expected.vertices)

  def test_build_convex_hull3(self):
    points = [Point(0, 0), Point(1, 0), Point(2, 0), Point(2, 1), Point(2, 2), Point(1, 2), Point(0, 2), Point(0, 1), Point(1, 1)]
    expected = Polygon([Point(2, 0), Point(2, 2), Point(0, 2), Point(0, 0)])
    actual = BuildConvexHull(points)
    self.assertEqual(len(actual.vertices), len(expected.vertices))
    self.assertCountEqual(actual.vertices, expected.vertices)

unittest.main(argv=[''], verbosity=2, exit=False)

test_build_convex_hull (__main__.TestNotebook) ... ok
test_build_convex_hull2 (__main__.TestNotebook) ... ok
test_build_convex_hull3 (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.007s

OK


## 2.3 Tangents to polygon

In [0]:
# Returns polygon vertices forming a tangent line to the polygon
def TangentsToPolygon(poly : Polygon, point : Point) -> Tuple[Point, Point] :
  # Get the polar angles for each point
  angles = []
  for p in poly.vertices:
    angles.append(tuple([p, Angle(point, p)]))
  
  List.sort(angles, key = lambda p: p[1])

  # Observe that the angles will not cover an arc > 180, but there may be circle wraparound
  if angles[-1][1] - angles[0][1] > 180:
    for i in range(len(angles)):
      if angles[i][1] > 180:
        break;
      angles[i] = tuple([angles[i][0], angles[i][1] + 360])
    List.sort(angles, key = lambda p: p[1]);
    # Failure indictates the point might be inside the polygon.
    assert angles[-1][1] - angles[0][1] < 180;

  return angles[-1][0], angles[0][0]

In [66]:
class TestNotebook(unittest.TestCase):

  def test_tangent(self):
    poly = Polygon([Point(23, 16), Point(-21, -12), Point(-7, -24), Point(18, -25)])

    self.assertTupleEqual(TangentsToPolygon(poly, Point(-7, 12)), tuple([Point(23, 16), Point(-21, -12)]))

    self.assertTupleEqual(TangentsToPolygon(poly, Point(24, 0)), tuple([Point(18, -25), Point(23, 16)]))

    self.assertTupleEqual(TangentsToPolygon(poly, Point(-12, -5)), tuple([Point(23, 16), Point(-21, -12)]))

    self.assertTupleEqual(TangentsToPolygon(poly, Point(-13, -25)), tuple([Point(-21,-12), Point(18, -25)]))

    self.assertTupleEqual(TangentsToPolygon(poly, Point(-9, 16)), tuple([Point(23, 16), Point(-21, -12)]))

  def test_tangent2(self):
    poly = Polygon([Point(-19, 20), Point(-23, 17), Point(-22, -16), Point(-10, -18), Point(-4, -17), Point(2, -12), Point(12, 1), Point(13, 3), Point(8, 14), Point(2, 19)])

    self.assertTupleEqual(TangentsToPolygon(poly, Point(-16, 20)), tuple([Point(2, 19), Point(-19, 20)]))

    self.assertTupleEqual(TangentsToPolygon(poly, Point(17, 7)), tuple([Point(2, -12), Point(2, 19)]))

    self.assertTupleEqual(TangentsToPolygon(poly, Point(-10, -23)), tuple([Point(-22, -16), Point(2, -12)]))

    self.assertTupleEqual(TangentsToPolygon(poly, Point(-24, 8)), tuple([Point(-23, 17), Point(-22, -16)]))

    self.assertTupleEqual(TangentsToPolygon(poly, Point(19, -18)), tuple([Point(-10, -18), Point(13, 3)]))

unittest.main(argv=[''], verbosity=2, exit=False)

test_tangent (__main__.TestNotebook) ... ok
test_tangent2 (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


## 2.4 Union of Convex Polygons

# Week 3
## Plane Sweep

In [0]:
def LineIntersection(l1 : Line, l2 : Line) -> Point {
  return None;
}

In [0]:
class TestNotebook(unittest.TestCase):

  def test_intersects(self):
    self.AssertEqual(LineIntersection(Line(Point(0,0), Point(0,1)), Line(Point(0,2), Point(0,3)), None)

unittest.main(argv=[''], verbosity=2, exit=False)